In [1]:
import os
from pathlib import Path

import pandas as pd

cwd = os.getcwd()
if cwd.endswith('notebook/PG'):
    os.chdir('../..')
    cwd = os.getcwd()

from src.pg_modelling.ligand_utils import sanitize_ligand_name

In [2]:
data_folder = Path(os.path.expanduser('~')) / 'Documents' / 'PG_modelling' / 'ZoocinA_O54308'
assert data_folder.is_dir()

In [13]:
pg_s_zooepidemicus = pd.concat(
    [
        pd.read_excel(
            data_folder / 'S_zooepidemicus_cell_wall_library.xlsx',
            sheet_name=0,
        ),
        pd.read_excel(
            data_folder / 'S_zooepidemicus_cell_wall_library.xlsx',
            sheet_name=1,
        )
    ],
    ignore_index=True,
)
pg_s_zooepidemicus['ligand_id'] = pg_s_zooepidemicus['Name'].apply(sanitize_ligand_name)
pg_s_zooepidemicus = pg_s_zooepidemicus.set_index('ligand_id')
print(f'N = {len(pg_s_zooepidemicus):,}')
pg_s_zooepidemicus.head()

N = 483


,Name,Synonym,Formula,Monoisotopic Mass,Modifications,Degree Amidation,Degree Acetylation,Ontology,PGN Units,Glycan Units,...,Peptide,SMILES,INCHIKEY,clogP,RT,[M+H]+,[M+Na]+,[M+K]+,[M+2H]2+,[M+3H]3+
ligand_id,,,,,,,,,,,,,,,,,,,,,
K-3-NH2-AAA,None-K[3-NH2-AAA],None--Lys[3--NH2--Ala.Ala.Ala],C15H29N5O5,359.21687,EPase P2,0,0,P3--S3,1,0,...,K[3-NH2-AAA],CC(N)C(=O)NC(C)C(=O)NC(C)C(=O)NCCCCC(N)C(=O)O,CDJMGOXZPXJIPZ-UHFFFAOYSA-N,-1.9586,0,360.22415,382.20609,398.18003,180.61572,120.74624
qK-3-NH2-AAA,None-qK[3-NH2-AAA],None--γ-isoGln.Lys[3--NH2--Ala.Ala.Ala],C20H37N7O7,487.27545,EPase P1,1,0,P3--S3,1,0,...,qK[3-NH2-AAA],CC(N)C(=O)NC(C)C(=O)NC(C)C(=O)NCCCCC(NC(=O)CCC...,KAHXFTOPUVCJRZ-UHFFFAOYSA-N,-3.2083,0,488.28273,510.26467,526.23861,244.64501,163.43243
AqK-3-NH2-AAA,None-AqK[3-NH2-AAA],None--Ala.γ-isoGln.Lys[3--NH2--Ala.Ala.Ala],C23H42N8O8,558.31256,Amidase,1,0,P3--S3,1,0,...,AqK[3-NH2-AAA],CC(N)C(=O)NC(C)C(=O)NC(C)C(=O)NCCCCC(NC(=O)CCC...,IANYAHOLBSYEDC-UHFFFAOYSA-N,-3.7036,0,559.31984,581.30178,597.27572,280.16356,187.11147
K-3-NH2-AA,None-K[3-NH2-AA],None--Lys[3--NH2--Ala.Ala],C12H24N4O4,288.17976,EPase P2,0,0,P3--S2,1,0,...,K[3-NH2-AA],CC(N)C(=O)NC(C)C(=O)NCCCCC(N)C(=O)O,JWCZPEHTCFTHJE-UHFFFAOYSA-N,-1.4633,0,289.18704,311.16898,327.14292,145.09716,97.06720
qK-3-NH2-AA,None-qK[3-NH2-AA],None--γ-isoGln.Lys[3--NH2--Ala.Ala],C17H32N6O6,416.23833,EPase P1,1,0,P3--S2,1,0,...,qK[3-NH2-AA],CC(N)C(=O)NC(C)C(=O)NCCCCC(NC(=O)CCC(N)C(N)=O)...,LSONAEWDWVGBIY-UHFFFAOYSA-N,-2.7130,0,417.24561,439.22755,455.20149,209.12645,139.75339


In [14]:
subset_ids = [
    'AqKAA',
    'NAG-NAM',
    'NAG-NAM-AqKAA',
    'NAG-NAM-AqKAA-3-NH2-AA',
    'NAG-NAM-AqKAA-3-NH2-AAA',
    'KA-3-NH2-AA-3br-4-AqKA',
    'KA-3-NH2-AAA-3br-4-AqKA',
]
pg_subset = pg_s_zooepidemicus.loc[subset_ids].rename(columns={
    'SMILES': 'smiles' 
})[['smiles']]
pg_subset
pg_subset.to_csv(data_folder / 'ligands.csv')
print(f'N = {len(pg_subset):,}')
pg_subset

N = 7


,smiles
ligand_id,
AqKAA,CC(N)C(=O)NC(CCC(=O)NC(CCCCN)C(=O)NC(C)C(=O)NC...
NAG-NAM,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...
NAG-NAM-AqKAA,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...
NAG-NAM-AqKAA-3-NH2-AA,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...
NAG-NAM-AqKAA-3-NH2-AAA,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...
KA-3-NH2-AA-3br-4-AqKA,CC(N)C(=O)NC(CCC(=O)NC(CCCCN)C(=O)NC(C)C(=O)NC...
KA-3-NH2-AAA-3br-4-AqKA,CC(N)C(=O)NC(CCC(=O)NC(CCCCN)C(=O)NC(C)C(=O)NC...
